In [1]:
import numpy as np
import sys
sys.path.append('../general')
from pool import Pool
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from collections import Counter
from metric import metric

In [2]:
pool = Pool('../data')

In [6]:
def test():
    train_pool, test_pool = pool.train_test_split()
    train_features = np.concatenate(
        (train_pool.features, np.reshape(train_pool.positions, (-1, 1))),
        axis=1
    )
    test_features = np.concatenate(
        (test_pool.features, np.reshape(test_pool.positions, (-1, 1))),
        axis=1
    )
    need_vertical_labels_train = np.array([
        1 if label == 3 else 0
        for label in train_pool.classification_labels
    ])
    need_vertical_labels_test = np.array([
        1 if label == 3 else 0
        for label in test_pool.classification_labels
    ])

    need_vertical_model = LogisticRegression()
    need_vertical_model.fit(train_features, need_vertical_labels_train)
    need_vertical = need_vertical_model.predict(test_features)

    positive_train_features = train_features[np.array(train_pool.classification_labels) == 3]
    positive_train_target = train_pool.targets[np.array(train_pool.classification_labels) == 3]

    postition_score_model = xgb.XGBRegressor()
    postition_score_model.fit(positive_train_features, positive_train_target)

    prediction = []
    for features, need_vertical_bool in zip(test_features, need_vertical):
        if not need_vertical_bool:
            prediction.append(100)
        else:
            max_score = -100
            best_position = -100
            for position in test_pool.POSITIONS:
                new_score = postition_score_model.predict([list(features)[:-1] + [position]])[0]
                if new_score > max_score:
                    max_score = new_score
                    best_position = position
            prediction.append(best_position)
    return metric(prediction, test_pool.positions, test_pool.targets, test_pool.probas)

In [7]:
results = [test() for i in range(100)]

In [8]:
np.mean(results)

-0.0013959389985173166

In [9]:
np.std(results)

0.00837579537187419

In [10]:
results

[0.0020257822163923284,
 -0.004068537727718805,
 0.007653224705637274,
 0.0056002487968361886,
 -0.015130559278948703,
 -0.0073836354783455285,
 0.006975416870619118,
 -0.0398158335475041,
 0.0030803867507852916,
 0.0036065758325915273,
 -0.0003487648234013926,
 0.0026163871762828273,
 0.000510057797142641,
 0.004252853710454962,
 0.0007780944752081662,
 -0.001561648014472906,
 -0.002299496954467789,
 0.003969255754882425,
 0.0001859647298928746,
 0.005690154058280687,
 -0.005877287511056444,
 -0.00022228821934383445,
 -0.04901391587971903,
 0.00017400670398030028,
 0.0036882552818810017,
 0.000979104607055473,
 -0.009606406764351769,
 -0.005715364894409975,
 -0.003291282668671489,
 -0.002085084440032425,
 0.0032012677092968073,
 -0.022435190641365278,
 0.0021301674723451405,
 -0.0012719857758875224,
 -0.0008151921903931775,
 -0.0009340035374503811,
 0.00572759215166137,
 -0.009827239044136985,
 0.002304278426380904,
 0.00400459336706308,
 -1.1032873320223073e-05,
 -0.00097029248273085